# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 25 2022 12:07PM,242659,10,8321591,Yusen – 3D Matrix,Released
1,Jul 25 2022 11:40AM,242656,19,8320971,"Innogenix, LLC",Executing
2,Jul 25 2022 11:40AM,242656,19,8320972,"Innogenix, LLC",Released
3,Jul 25 2022 11:40AM,242656,19,8320973,"Innogenix, LLC",Executing
4,Jul 25 2022 11:40AM,242656,19,8320974,"Innogenix, LLC",Executing
5,Jul 25 2022 11:40AM,242656,19,8320975,"Innogenix, LLC",Executing
6,Jul 25 2022 11:40AM,242656,19,8320976,"Innogenix, LLC",Executing
7,Jul 25 2022 11:40AM,242656,19,8320977,"Innogenix, LLC",Executing
8,Jul 25 2022 11:40AM,242656,19,8320978,"Innogenix, LLC",Executing
9,Jul 25 2022 11:40AM,242656,19,8320979,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,242654,Released,2
31,242655,Released,8
32,242656,Executing,12
33,242656,Released,10
34,242659,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242652,NaN,1.0
242654,7.0,2.0
242655,NaN,8.0
242656,12.0,10.0
242659,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242576,0.0,1.0
242579,0.0,1.0
242581,0.0,1.0
242583,0.0,1.0
242591,3.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242576,0,1
242579,0,1
242581,0,1
242583,0,1
242591,3,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242576,0,1
1,242579,0,1
2,242581,0,1
3,242583,0,1
4,242591,3,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242576,,1
1,242579,,1
2,242581,,1
3,242583,,1
4,242591,3,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 25 2022 12:07PM,242659,10,Yusen – 3D Matrix
1,Jul 25 2022 11:40AM,242656,19,"Innogenix, LLC"
23,Jul 25 2022 11:39AM,242655,10,"Snap Medical Industries, LLC"
31,Jul 25 2022 11:36AM,242654,10,Beach Products Inc
40,Jul 25 2022 11:36AM,242581,10,Eye Pharma Inc
41,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc."
42,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.
43,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC"
45,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated"
46,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 25 2022 12:07PM,242659,10,Yusen – 3D Matrix,,1
1,Jul 25 2022 11:40AM,242656,19,"Innogenix, LLC",12,10
2,Jul 25 2022 11:39AM,242655,10,"Snap Medical Industries, LLC",,8
3,Jul 25 2022 11:36AM,242654,10,Beach Products Inc,7,2
4,Jul 25 2022 11:36AM,242581,10,Eye Pharma Inc,,1
5,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",,1
6,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,,1
7,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",,2
8,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",,1
9,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",4,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 12:07PM,242659,10,Yusen – 3D Matrix,1,
1,Jul 25 2022 11:40AM,242656,19,"Innogenix, LLC",10,12
2,Jul 25 2022 11:39AM,242655,10,"Snap Medical Industries, LLC",8,
3,Jul 25 2022 11:36AM,242654,10,Beach Products Inc,2,7
4,Jul 25 2022 11:36AM,242581,10,Eye Pharma Inc,1,
5,Jul 25 2022 11:33AM,242652,21,"NBTY Global, Inc.",1,
6,Jul 25 2022 11:19AM,242651,10,Eywa Pharma Inc.,1,
7,Jul 25 2022 11:04AM,242644,15,"Carwin Pharmaceutical Associates, LLC",2,
8,Jul 25 2022 11:03AM,242643,10,"Senseonics, Incorporated",1,
9,Jul 25 2022 11:00AM,242637,15,"Brookfield Pharmaceuticals, LLC",20,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 12:07PM,242659,10,Yusen – 3D Matrix,1,
1,Jul 25 2022 11:40AM,242656,19,"Innogenix, LLC",10,12
2,Jul 25 2022 11:39AM,242655,10,"Snap Medical Industries, LLC",8,
3,Jul 25 2022 11:36AM,242654,10,Beach Products Inc,2,7
4,Jul 25 2022 11:36AM,242581,10,Eye Pharma Inc,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 12:07PM,242659,10,Yusen – 3D Matrix,1.0,NaN
1,Jul 25 2022 11:40AM,242656,19,"Innogenix, LLC",10.0,12.0
2,Jul 25 2022 11:39AM,242655,10,"Snap Medical Industries, LLC",8.0,NaN
3,Jul 25 2022 11:36AM,242654,10,Beach Products Inc,2.0,7.0
4,Jul 25 2022 11:36AM,242581,10,Eye Pharma Inc,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 25 2022 12:07PM,242659,10,Yusen – 3D Matrix,1.0,0.0
1,Jul 25 2022 11:40AM,242656,19,"Innogenix, LLC",10.0,12.0
2,Jul 25 2022 11:39AM,242655,10,"Snap Medical Industries, LLC",8.0,0.0
3,Jul 25 2022 11:36AM,242654,10,Beach Products Inc,2.0,7.0
4,Jul 25 2022 11:36AM,242581,10,Eye Pharma Inc,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4609777,122.0,54.0
15,970549,60.0,30.0
16,242634,1.0,0.0
19,727865,13.0,15.0
21,485282,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4609777,122.0,54.0
1,15,970549,60.0,30.0
2,16,242634,1.0,0.0
3,19,727865,13.0,15.0
4,21,485282,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,122.0,54.0
1,15,60.0,30.0
2,16,1.0,0.0
3,19,13.0,15.0
4,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,122.0
1,15,Released,60.0
2,16,Released,1.0
3,19,Released,13.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Executing,54.0,30.0,0.0,15.0,1.0
Released,122.0,60.0,1.0,13.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Executing,54.0,30.0,0.0,15.0,1.0
1,Released,122.0,60.0,1.0,13.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Executing,54.0,30.0,0.0,15.0,1.0
1,Released,122.0,60.0,1.0,13.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()